In [1]:
import pandas as pd
import requests

## Data collection

Se realiza la lectura de los objetos directamente desde el github de felipeares.

In [2]:
url = 'https://raw.githubusercontent.com/felipeares/legalbot-desafio-objetos/master/objetos.txt'
page = requests.get(url)
#Se identificó visualmente que el separador de los objetos corresponde a los caracteres \n\n
df_corpus = pd.DataFrame(page.text.split('\n\n'), columns = ['objeto'])

In [3]:
df_corpus.head()

,objeto
0,"CONSTRUCCIÓN, ALQUILER DE OTROS TIPOS DE MAQUI..."
1,el servicio de turismo por todo el territorio ...
2,SALON DE BELLEZA.
3,"CURSOS ONLINE DE MUSICA, COCINA, CLASES DE MUS..."
4,"el ejercicio de la actividad comercial,importa..."


## Data Preprocessing

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcartesf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcartesf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jcartesf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import string  #Se obtendrá los signos de puntuación
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, SnowballStemmer


In [6]:
#Método limpieza inicial y tokenización
def get_tokens_clean(objeto):
    #Se deja el texto en minúscula
    objeto = objeto.lower()
    
    #Elimina tildes
    a,b = 'áéíóúü','aeiouu',
    trans = str.maketrans(a,b)
    objeto = objeto.translate(trans)
    
    #Elimina caracteres que no sean palabras
    filtered_punctuation =  objeto.translate({ord(c): None for c in string.punctuation})
    
    #tokeniza
    tokens = nltk.word_tokenize(filtered_punctuation)
    
    #Se eliminan stopwords (articulos, preposiciones, etc.)
    no_stopwords = [word for word in tokens if word not in stopwords.words('spanish')]
       
    return no_stopwords

#Función de derivacion 
def stem_words(tokens):
    stemmer_spanish = SnowballStemmer('spanish')
    stems = []
    for word in tokens:
        stem = stemmer_spanish.stem(word)
        stems.append(stem)
    return stems

#Elimina palabas menor a tres caracteres
def delete_characteres(tokens):
    return  [word for word in tokens if len(word)>3]

#Lematización de verbos. Por ahora sólo palabras en inglés.
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

#Se eliminan palabras derivadas con alta frecuencia, ya que sesga los grupos.
def replace_word_frequence(tokens):
    replace_word = []
    for word in tokens:
        word = word.replace('activ','').replace('servici','')
        if word == '': continue
        replace_word.append(word)
    return replace_word

In [7]:
from gensim.corpora import Dictionary

df_corpus['objeto_tokens'] = df_corpus['objeto'].map(get_tokens_clean)
df_corpus['objeto_tokens'] = df_corpus['objeto_tokens'].map(delete_characteres) 
df_corpus['objeto_stemm']  = df_corpus['objeto_tokens'].map(stem_words)
df_corpus['objeto_stemm'] = df_corpus['objeto_stemm'].map(replace_word_frequence)

c:\users\jcartesf\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Create dictionary

In [8]:
#Se crea diccionario
dct = Dictionary(df_corpus['objeto_stemm'].tolist()))

## Data exploration

In [13]:
'''import matplotlib.pyplot as plt
flat_list = [word for idx,array_word in df_corpus.iterrows() for word in array_word['objeto_stemm']]
word_group = pd.DataFrame(flat_list,index=flat_list,columns=['word'])['word']
word_group_count = word_group.value_counts()
## VENTAS POR MES ####
plt.figure(figsize=(15, 15))
plt.ion()

plt.plot(word_group_count[word_group_count>300], 'b')  # Dibujamos una línea recta azul
plt.xlabel('palabras')  # Ponemos etiqueta al eje x
plt.ylabel(u'Cantidad')  # Ponemos etiqueta al eje y
plt.xticks(rotation=90)
plt.show()'''